In [1]:
#@title Data retrieval
import os, requests

fname = "stringer_spontaneous.npy"
url = "https://osf.io/dpqaj/download"

if not os.path.isfile(fname):
    try:
        r = requests.get(url)
    except requests.ConnectionError:
    print("!!! Failed to download data !!!")
    else:
        if r.status_code != requests.codes.ok:
          print("!!! Failed to download data !!!")
        else:
            with open(fname, "wb") as fid:
                fid.write(r.content)

In [3]:
import numpy as np

In [5]:
dat=np.load('stringer_spontaneous.npy',allow_pickle=True)

In [10]:
dat.item()['sresp'].shape

(11983, 7018)

In [11]:
from sklearn.decomposition import PCA

In [14]:
pca=PCA(n_components=10)
pcs=pca.fit_transform(dat.item()['sresp'].T)
print(pcs.shape)

(7018, 10)


In [17]:
x=[]
y=[]
for j in range(0,5000):
    x.append(pcs[j,:].flatten())
    y.append(pcs[j+1,:].flatten())

In [20]:
x_test=[]
y_test=[]
for j in range(5000,7000):
    x_test.append(pcs[j,:].flatten())
    y_test.append(pcs[j+1,:].flatten())

In [18]:
x=np.array(x)
y=np.array(y)
print(x.shape)

(5000, 10)


In [19]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x, y)
reg.score(x, y)

0.470741425670049

In [21]:
reg.score(x_test,y_test)

0.47756974888944087

In [28]:
y_pred=reg.predict(x_test)

In [27]:
from sklearn.metrics import explained_variance_score

print(explained_variance_score(pcs[5001:7001],y_test))

1.0
